In [1]:
# I don't think all of these are needed as this was part of a larger file when I got it.  However, I didn't have time to look
# through and remove the ones that are unnecessary.
import os
import urllib.request
import shutil
import re
from pathlib import Path

from os.path import expanduser
import pandas as pd
import glob
import io
from bs4 import BeautifulSoup

import sys
import unicodedata
import requests

import pandas as pd

In [2]:
# Code to download index files. This code is from stackoverflow - https://stackoverflow.com/questions/68131406/downloading-files-from-sec-gov-via-edgar-using-python-3-9
# The key is to specify a header which signals to the SEC server that you are a real user and not a bot. Edit personal info
# below to include your company name and a real email address.

def download(year):
    for qtr in range(1, 5):
        url = f"https://www.sec.gov/Archives/edgar/full-index/{year}/QTR{qtr}/master.idx"
        response = requests.get(url, headers={'User-Agent': 'University of Southern California payneman@my.usc.edu'}, stream=True)
        print(url)
        response.raise_for_status()
        down_direct = os.path.join(Path().absolute(), 'Indexes')

        with open(f'{down_direct}/master{year}QTR{qtr}.idx', 'wb') as f:
            f.write(response.content)


In [3]:

def addEntry(cik, ftype, date, path, df):
    return df.append({"CIK": cik, "Filing Type": ftype, "Date": date, "File": path}, ignore_index = True)

In [4]:
# Function to convert file path into clickable form.
def fun(path):
    
    # returns the final component of a url
    f_url = os.path.basename(path)
      
    # convert the url into link
    return '<a href="{}">{}</a>'.format(path, f_url)

In [5]:
# From Bochkay textbook (12.2.2), modified to include User Agent header, which circumvents 403 Error Code
# Make sure to edit company name and email address in "response = request."" line of code

# Places in this cell to edit: 
# Bottom 4 lines of code --> Specify regex file you want (13F in this case), file locations, and years
# Can adjust quarters at "for qtr in range (1,5)"
# Can adjust "count" to test code before running it for full year

def get_files(start_year:int, end_year:int, 
              start_qtr:int, end_qtr:int,
              reform:str, 
              inddirect:str, odirect:str, cik:str):
    """
    Downloads SEC filings for specific companies
    start_year -> First Year to download
    end_year -> Last Year to download
    start_qtr -> qtr of first yr
    end_qtr -> qtr of last yr
    reform -> Regex to specify forms to be downloaded
    inddirect -> Directory containing index files
    odirect -> Directory the filings will be downloaded to
    cik -> cik of company
    """
    
    print('Downloading Filings for CIK#' + cik)
    
    global out_df
    
    # Regex to identify the form to download.
    re_formtype = re.compile(reform, re.IGNORECASE)
    # Regex to extract file name information 
    # from a line
    re_fullfilename = re.compile(r"\|(edgar/data.*\/([\d-]+\.txt))", re.IGNORECASE)
    
    re_date = re.compile(r"\|(20[0-9][0-9])-(0[1-9]|1[0-2])-(0[1-9]|[12][0-9]|3[01])\|", re.IGNORECASE)

    re_cik = '(' + cik + '\|)'
    
    re_cik = re.compile(re_cik, re.IGNORECASE)
    
    #loop through the index files based on year
    for year in range(start_year, end_year+1):
        #check whether the directory exists and create one if it does not.
        download_path = os.path.join(odirect)
            
        start = 1 
        end = 5
        
        if year == start_year:
            start = start_qtr
        else:
            start = 1
            
        if year == end_year:
            end = end_qtr+1
        else:
            end = 5
            
        for qtr in range(start,end):
            #name of index file to be read.
            dl_file = os.path.join(inddirect, 'master' + str(year) + 'QTR' + str(qtr) + '.idx')
        
            # open the index file
            with open(dl_file, 'rb') as f:
                # set a counter called count to 1. Note 
                # that the counter will only be incremented 
                # after it downloads a file.
                    count=1
                # loop through each line in the index file, assigning to a variable called line
                    for bline in f:
                        try:
                            line = bline.decode()
                    # Only download a file if the counter is less than 5 (to test code). Remove this if statement if you want
                    # to download all the files for the time period or adjust if you want to expand sample.
                            if count<5:
                            # Check to see if the line matches the cik
                                rematch=re.search(re_cik,line)
                                #If there is a match then download the filing
                                if rematch:
                                    str_cik = str(rematch.group(1).replace("|", ""))
                                    # Check if form type mattches now
                                    rematch = re.search(re_formtype, line)
                                    if rematch:
                                        ftype = str(rematch.group(1).replace("|", "").replace("/A", ""))
                                        date = str("-".join(re.search(re_date, line).groups()).replace("|", ""))
                                        # The following line searches for filename information. 
                                        # The first grouping will contain the location and filename of the file to be 
                                        # downloaded. The second grouping will contain just contain the filename o
                                        matches = re.search(re_fullfilename, line)
                                        if matches:
                                            # Construct the url to for retrieving the filing 
                                            url = str('https://www.sec.gov/Archives/') + str(matches.group(1))
                                            # Create the filename to download the file to.
                                            download_path = os.path.join(odirect, ftype)
                                            if not os.path.exists(download_path):
                                                os.makedirs(download_path)
                                            outfile = os.path.join(download_path, 
                                                                   str(matches.group(2)))
                                            rel_path = os.path.relpath(outfile)
                                            # Check to make sure the file hasn't already been downloaded                    
                                            if not (os.path.isfile(outfile) and os.access(outfile, os.R_OK)):
                                                # Print the name of the 
                                                # file to be downloaded.
                                                print("Downloading:"+str(outfile),end='\n')
                                                response = requests.get(url, headers={'User-Agent': 'University of Southern California payneman@my.usc.edu'}, stream=True)
                                                response.raise_for_status()
                                                with open(f'{outfile}', 'wb') as f:
                                                    f.write(response.content)   
                                                    
                                                # add row to dataframe here
                                                out_df = addEntry(str_cik, ftype, date, rel_path, out_df)
                                                count += 1
                        except UnicodeDecodeError:
                            pass
                        except KeyboardInterrupt:
                            raise
                        except:
                            pass
    print('Downloading of Filings Complete for CIK#'+cik ,end='\n')
    return
                                                       
# Specify, in regular expression format, the filing
# you are looking for.  Following is the for 10-k.
# reform='(\|10-?k(sb|sb40|405)?\s*\|)'

# RegEx for 10-K 8-K and 10-Q filings
reform='(\|10-?K\/*A*|8-?K\/*A*|10-?Q\/*A*\|)'

# Specify location of the index files.
indirect = os.path.join(Path().absolute(), 'Indexes')

# Specify where to download filings to
odirect = os.path.join(Path().absolute(), 'Out_Files')

# Execute the get filings function
# get_files(2006, 2006, reform, inddirect, odirect)

In [6]:
df = pd.read_csv("input_data.csv")

In [7]:
df = df[['cikmissing', 'AuthorName', 'company', 'PERMNO', 'TICKER', 'date', 'cik']]

In [8]:
df = df[df['cikmissing'] == 0]

In [9]:
df["cik"] = pd.to_numeric(df["cik"], downcast="integer")
df["cik"] = df["cik"].astype(str)

In [10]:
df["date"] = pd.to_datetime(df["date"])

In [11]:
df

,cikmissing,AuthorName,company,PERMNO,TICKER,date,cik
0,0,Citron Research,ESCALA GROUP INC,79167,ESCL,2006-01-04,895516
1,0,Citron Research,CHINA TECHNOLOGY DEVELOP GP COR,84343,CTDC,2006-02-09,1027454
2,0,Citron Research,HOUSEVALUES INC,90489,SOLD,2006-02-21,1298978
3,0,Citron Research,FAIRFAX FINL HOLDINGS LTD,89627,FFH,2006-03-27,915191
4,0,Manuel P. Asensio,CROCS INC,91090,CROX,2006-03-29,1334036
...,...,...,...,...,...,...,...
1901,0,J Capital Research Limi,CBAK Energy Technology Inc.,91269,CBAT,2020-12-18,1117171
1902,0,Hindenburg Research,MicroVision Inc.,83885,MVIS,2020-12-22,65770
1903,0,Marcus Aurelius Value,Penumbra Inc,15694,PEN,2020-12-28,1321732
1904,0,Kerrisdale Capital Mana,fuboTV Inc.,19893,FUBO,2020-12-30,1484769


In [12]:
min_date = min(df["date"])
max_date = max(df["date"])

In [13]:
max_date = max_date.replace(year=max_date.year+1)

In [14]:
start_yr = min_date.year
end_yr = max_date.year

In [15]:
for i in range(start_yr,end_yr+1):
    download(i)

https://www.sec.gov/Archives/edgar/full-index/2006/QTR1/master.idx
https://www.sec.gov/Archives/edgar/full-index/2006/QTR2/master.idx
https://www.sec.gov/Archives/edgar/full-index/2006/QTR3/master.idx
https://www.sec.gov/Archives/edgar/full-index/2006/QTR4/master.idx
https://www.sec.gov/Archives/edgar/full-index/2007/QTR1/master.idx
https://www.sec.gov/Archives/edgar/full-index/2007/QTR2/master.idx
https://www.sec.gov/Archives/edgar/full-index/2007/QTR3/master.idx
https://www.sec.gov/Archives/edgar/full-index/2007/QTR4/master.idx
https://www.sec.gov/Archives/edgar/full-index/2008/QTR1/master.idx
https://www.sec.gov/Archives/edgar/full-index/2008/QTR2/master.idx
https://www.sec.gov/Archives/edgar/full-index/2008/QTR3/master.idx
https://www.sec.gov/Archives/edgar/full-index/2008/QTR4/master.idx
https://www.sec.gov/Archives/edgar/full-index/2009/QTR1/master.idx
https://www.sec.gov/Archives/edgar/full-index/2009/QTR2/master.idx
https://www.sec.gov/Archives/edgar/full-index/2009/QTR3/master

In [16]:
def get_quarter(date):
    return (date.month-1) // 3 + 1

In [17]:
out_df = pd.DataFrame([], [], ["CIK", "Filing Type", "Date", "File"])

In [18]:
for index, row in df.iterrows():
    get_files(row["date"].year, row["date"].year+1, get_quarter(row["date"]), get_quarter(row["date"]), reform, indirect, odirect, row["cik"])

Downloading:/Users/alekseyvalouev/Desktop/edgar-data-gathering/Out_Files/10-Q/0000922423-06-000187.txt
Downloading:/Users/alekseyvalouev/Desktop/edgar-data-gathering/Out_Files/8-K/0000922423-06-000028.txt
Downloading:/Users/alekseyvalouev/Desktop/edgar-data-gathering/Out_Files/8-K/0000922423-06-000134.txt
Downloading:/Users/alekseyvalouev/Desktop/edgar-data-gathering/Out_Files/8-K/0000922423-06-000160.txt
Downloading:/Users/alekseyvalouev/Desktop/edgar-data-gathering/Out_Files/10-Q/0000922423-06-000670.txt
Downloading:/Users/alekseyvalouev/Desktop/edgar-data-gathering/Out_Files/8-K/0000922423-06-000744.txt
Downloading:/Users/alekseyvalouev/Desktop/edgar-data-gathering/Out_Files/8-K/0001157523-06-005891.txt
Downloading:/Users/alekseyvalouev/Desktop/edgar-data-gathering/Out_Files/8-K/0001157523-06-007084.txt
Downloading:/Users/alekseyvalouev/Desktop/edgar-data-gathering/Out_Files/8-K/0001157523-06-007548.txt
Downloading:/Users/alekseyvalouev/Desktop/edgar-data-gathering/Out_Files/8-K/000

In [19]:
out_df

,CIK,Filing Type,Date,File
0,895516,10-Q,2006-02-09,Out_Files/10-Q/0000922423-06-000187.txt
1,895516,8-K,2006-01-06,Out_Files/8-K/0000922423-06-000028.txt
2,895516,8-K,2006-01-30,Out_Files/8-K/0000922423-06-000134.txt
3,895516,8-K,2006-02-06,Out_Files/8-K/0000922423-06-000160.txt
4,895516,10-Q,2006-05-10,Out_Files/10-Q/0000922423-06-000670.txt
...,...,...,...,...
21433,1691421,10-Q,2021-08-11,Out_Files/10-Q/0001691421-21-000081.txt
21434,1691421,8-K,2021-07-26,Out_Files/8-K/0001691421-21-000072.txt
21435,1691421,8-K,2021-08-04,Out_Files/8-K/0001691421-21-000077.txt
21436,1691421,10-Q,2021-11-09,Out_Files/10-Q/0001691421-21-000094.txt


In [20]:
out_df = out_df.style.format({'File' : fun})

In [21]:
out_df = out_df.data

In [24]:
out_df.to_csv('out.csv', index=False)

In [23]:
out_df

,CIK,Filing Type,Date,File
0,895516,10-Q,2006-02-09,Out_Files/10-Q/0000922423-06-000187.txt
1,895516,8-K,2006-01-06,Out_Files/8-K/0000922423-06-000028.txt
2,895516,8-K,2006-01-30,Out_Files/8-K/0000922423-06-000134.txt
3,895516,8-K,2006-02-06,Out_Files/8-K/0000922423-06-000160.txt
4,895516,10-Q,2006-05-10,Out_Files/10-Q/0000922423-06-000670.txt
...,...,...,...,...
21433,1691421,10-Q,2021-08-11,Out_Files/10-Q/0001691421-21-000081.txt
21434,1691421,8-K,2021-07-26,Out_Files/8-K/0001691421-21-000072.txt
21435,1691421,8-K,2021-08-04,Out_Files/8-K/0001691421-21-000077.txt
21436,1691421,10-Q,2021-11-09,Out_Files/10-Q/0001691421-21-000094.txt
